In [2]:
# python

import csv
import json
from pathlib import Path
import requests

BASE_URL = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"



In [3]:
query = "((COVID-19) AND 2026 [pdat])"

def get_search_context(query: str) -> tuple[str, str]:

    params = {

    "db": "pmc",

    "term": query,

    "retmode": "json",

    "usehistory": "y",

    "retmax": 0,

    }

    response = requests.get(f"{BASE_URL}/esearch.fcgi", params=params, timeout=30)

    response.raise_for_status()

    data: dict = response.json().get("esearchresult", {})

    return data.get("webenv", None), data.get("querykey", None), data.get("count", None)

In [3]:
get_search_context(query=query)

('MCID_6997ca4c02f1a7a0a9022c2e', '1', '5192')

In [18]:
params = {

    "db": "pmc",

    "term": query,

    "retmode": "json",

    "usehistory": "y",

    "retmax": 0,

    }

response = requests.get(f"{BASE_URL}/esearch.fcgi", params=params, timeout=30)

response.raise_for_status()

data: dict = response.json().get("esearchresult", {})

In [19]:
data

{'count': '5192',
 'retmax': '0',
 'retstart': '0',
 'querykey': '1',
 'webenv': 'MCID_6997d0c15ab01606d3042c4a',
 'idlist': [],
 'translationset': [{'from': 'COVID-19',
   'to': '("COVID-19" OR "COVID19" OR "COVID-19"[MeSH Terms] OR "COVID-19 Vaccines" OR "COVID-19 Vaccines"[MeSH Terms] OR "COVID-19 serotherapy" OR "COVID-19 serotherapy"[MeSH Terms] OR "COVID-19 Nucleic Acid Testing" OR "covid-19 nucleic acid testing"[MeSH Terms] OR "COVID-19 Serological Testing" OR "covid-19 serological testing"[MeSH Terms] OR "COVID-19 Testing" OR "covid-19 testing"[MeSH Terms] OR "SARS-CoV-2" OR "SARSCoV2" OR "SARSCoV-2" OR "SARS-CoV2" OR "sars-cov-2"[MeSH Terms] OR "Severe Acute Respiratory Syndrome Coronavirus 2" OR "2019 NCOV" OR (("coronavirus"[MeSH Terms] OR "coronavirus" OR "COV" OR "NCOV") AND 2019/11/01[PDAT] : 3000/12/31[PDAT]))'}],
 'querytranslation': '("covid 19"[All Fields] OR "covid19"[All Fields] OR "covid 19"[MeSH Terms] OR "covid 19 vaccines"[All Fields] OR "covid 19 vaccines"[MeSH

In [25]:
params = {
    "db": "pmc",   # ← change here
    "term": query,
    "retmode": "json",
    "usehistory": "y",
    "retmax": 100,
}

response = requests.get(f"{BASE_URL}/esearch.fcgi", params=params, timeout=30)
response.raise_for_status()

data = response.json().get("esearchresult", {})

pmids = data.get("idlist", [])

print(pmids)

['12915591', '12916102', '12913732', '12913649', '12913897', '12912951', '12915721', '12915835', '12913034', '12914922', '12914071', '12914070', '12913041', '12913840', '12915807', '12915772', '12915886', '12914983', '12915769', '12915854', '12915924', '12912970', '12915787', '12916212', '12913009', '12915816', '12913901', '12865442', '12865458', '12865426', '12865439', '12865463', '12862823', '12915786', '12913627', '12913844', '12914031', '12913890', '12913784', '12913972', '12913777', '12913775', '12913937', '12913629', '12914058', '12913953', '12913989', '12915517', '12915328', '12914965', '12914916', '12913791', '12914473', '12915020', '12915027', '12914935', '12914957', '12914984', '12914028', '12913899', '12915340', '12914629', '12913623', '12915356', '12915080', '12915109', '12915164', '12915081', '12915171', '12915903', '12916117', '12915962', '12915914', '12915971', '12915944', '12916192', '12914663', '12913085', '12914503', '12915044', '12913380', '12914326', '12910234', '12

In [4]:
pmid = "12913732"

params = {
    "db": "pmc",
    "id": pmid,
    "retmode": "json"
}

response = requests.get(BASE_URL + "esummary.fcgi", params=params)
response.raise_for_status()

data = response.json()

In [5]:
data

{'header': {'type': 'esummary', 'version': '0.3'},
 'result': {'uids': ['12913732'],
  '12913732': {'uid': '12913732',
   'pubdate': '2026 Feb 17',
   'epubdate': '2026 Feb 17',
   'printpubdate': '',
   'source': 'Npj Viruses',
   'authors': [{'name': 'Yordanova IA', 'authtype': 'Author'},
    {'name': 'Prescott JB', 'authtype': 'Author'}],
   'title': 'Revisiting Ravn virus as the lesser known orthomarburgvirus.',
   'volume': '4',
   'issue': '1',
   'pages': '11',
   'articleids': [{'idtype': 'pmid', 'value': '41703278'},
    {'idtype': 'doi', 'value': '10.1038/s44298-026-00180-x'},
    {'idtype': 'pii', 'value': '10.1038/s44298-026-00180-x'},
    {'idtype': 'pmcid', 'value': 'PMC12913732'}],
   'fulljournalname': 'Npj viruses',
   'sortdate': '2026/02/17 00:00',
   'pmclivedate': '2026/02/19'}}}

In [41]:
def parse_pubmed_esummary(results: dict, pmid: str) -> dict:
    """
    Parse the results for one specific pmid
    From the pubmed esummary return
    Return the dictionary of needed data
    """

    article = results.get("result", {}).get(pmid)
    if not article:
        return {}

    # Authors
    authors = article.get("authors", [])
    author_names = [a.get("name", "") for a in authors if "name" in a]

    # Article IDs
    doi = ""
    pmcid = ""
    pubmed_id = ""

    for aid in article.get("articleids", []):
        if aid.get("idtype") == "doi":
            doi = aid.get("value", "")
        if aid.get("idtype") == "pmcid":
            pmcid = aid.get("value", "")
        if aid.get("idtype") == "pmid":
            pubmed_id = aid.get("value", "")

    # Dates
    pubdate = article.get("pubdate", "")
    epubdate = article.get("epubdate", "")
    printpubdate = article.get("printpubdate", "")
    sortdate = article.get("sortdate", "")
    pmc_live_date = article.get("pmclivedate", "")

    resolved_date = epubdate or pubdate or printpubdate or sortdate

    return {
        "uid": pmid,                 # the key used in esummary
        "pubmed_id": pubmed_id,      # the true PMID from articleids
        "title": article.get("title", ""),
        "journal": article.get("fulljournalname", ""),
        "journal_abbrev": article.get("source", ""),
        "pubdate": pubdate,
        "epubdate": epubdate,
        "printpubdate": printpubdate,
        "sortdate": sortdate,
        "pmc_live_date": pmc_live_date,
        "resolved_pubdate": resolved_date,
        "volume": article.get("volume", ""),
        "issue": article.get("issue", ""),
        "pages": article.get("pages", ""),
        "doi": doi,
        "pmcid": pmcid,
        "authors": "; ".join(author_names),
        "author_count": len(author_names),
        "publication_types": "; ".join(article.get("pubtype", [])),
        "language": "; ".join(article.get("lang", [])),
    }

In [42]:
parse_pubmed_esummary(data, pmid)

{'uid': '12913732',
 'pubmed_id': '41703278',
 'title': 'Revisiting Ravn virus as the lesser known orthomarburgvirus.',
 'journal': 'Npj viruses',
 'journal_abbrev': 'Npj Viruses',
 'pubdate': '2026 Feb 17',
 'epubdate': '2026 Feb 17',
 'printpubdate': '',
 'sortdate': '2026/02/17 00:00',
 'pmc_live_date': '2026/02/19',
 'resolved_pubdate': '2026 Feb 17',
 'volume': '4',
 'issue': '1',
 'pages': '11',
 'doi': '10.1038/s44298-026-00180-x',
 'pmcid': 'PMC12913732',
 'authors': 'Yordanova IA; Prescott JB',
 'author_count': 2,
 'publication_types': '',
 'language': ''}